In [ ]:
!pip install -q \
langchain==0.1.20 \
langchain-community==0.0.38 \
langchain-openai==0.1.7 \
faiss-cpu \
fastapi \
pydantic \
tiktoken


In [ ]:
import langchain
import faiss
import fastapi
import tiktoken

print("Task 1 successful: Environment is stable")


In [ ]:
#Create a Mock LLM Function
def mock_llm(context: str, question: str) -> str:
    """
    Simulates an LLM response using retrieved context.
    """
    return f"""
Answer based on company documents:

Context:
{context}

Question:
{question}

Final Answer:
Based on the provided documentation, please follow the relevant steps described above.
""".strip()


In [ ]:
test_context = "Users can reset their password from the Settings page."
test_question = "How do I reset my password?"

print(mock_llm(test_context, test_question))


In [ ]:
#Create Support Knowledge
support_documents = """
ACCOUNT & LOGIN SUPPORT

1. Password Reset
Users can reset their password by navigating to the Account Settings page.
Click on 'Forgot Password', enter the registered email address, and follow the reset link sent via email.

2. Account Locked
Accounts may be temporarily locked after multiple failed login attempts.
Users should wait 30 minutes before trying again or contact customer support for immediate assistance.

3. Two-Factor Authentication
Two-factor authentication can be enabled or disabled from the Security section of Account Settings.
A verification code will be sent to the registered mobile number or email.

BILLING & PAYMENTS

4. Payment Failure
Payments may fail due to insufficient balance, expired cards, or bank restrictions.
Users are advised to verify card details or contact their bank.

5. Refund Policy
Refunds are processed within 5–7 business days after approval.
Approved refunds are credited back to the original payment method.

GENERAL SUPPORT

6. Updating Personal Information
Users can update their name, address, and contact details from the Profile section.

7. Contacting Support
Customers can contact support via email or live chat.
Support is available Monday to Friday, 9 AM to 6 PM.
"""


In [ ]:
print("Total characters in support knowledge:", len(support_documents))
print("\nPreview:\n")
print(support_documents[:300])


EMBEDDINGS + FAISS

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS


In [ ]:
from langchain.schema import Document

documents = [Document(page_content=support_documents)]
print("Number of documents:", len(documents))


In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=300,
    chunk_overlap=50
)

chunks = text_splitter.split_documents(documents)

print("Number of chunks created:", len(chunks))
print("\nSample chunk:\n")
print(chunks[0].page_content)


In [ ]:
import numpy as np
import hashlib

def mock_embedding(text: str, dim: int = 128):
    """
    Converts text into a deterministic numeric vector.
    This simulates real embeddings for development.
    """
    hash_digest = hashlib.sha256(text.encode()).digest()
    vector = np.frombuffer(hash_digest, dtype=np.uint8)[:dim]
    return vector.astype("float32")


In [ ]:
vectors = []
texts = []

for chunk in chunks:
    texts.append(chunk.page_content)
    vectors.append(mock_embedding(chunk.page_content))

vectors = np.vstack(vectors)

print("Vector shape:", vectors.shape)


In [ ]:
import faiss

dimension = vectors.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(vectors)

print("Total vectors in FAISS index:", index.ntotal)


In [ ]:
faiss.write_index(index, "support_faiss.index")
print("FAISS index saved successfully")


RAG = Retrieve → Augment → Generate

In [ ]:
def embed_question(question: str):
    return mock_embedding(question)


In [ ]:
def retrieve_relevant_chunks(question: str, top_k: int = 3):
    question_vector = embed_question(question).reshape(1, -1)

    distances, indices = index.search(question_vector, top_k)

    retrieved_chunks = [texts[i] for i in indices[0]]
    return retrieved_chunks


In [ ]:
def build_context(retrieved_chunks):
    return "\n\n".join(retrieved_chunks)


In [ ]:
def rag_answer(question: str):
    retrieved_chunks = retrieve_relevant_chunks(question)
    context = build_context(retrieved_chunks)

    answer = mock_llm(context=context, question=question)
    return answer


In [ ]:
test_question = "How can I reset my password?"

response = rag_answer(test_question)
print(response)


In [ ]:
print("Enterprise Support Chatbot is running.")
print("Type your question. Type 'exit' to stop.\n")

while True:
    user_question = input("You: ")

    if user_question.lower().strip() == "exit":
        print("Chatbot stopped.")
        break

    print("\nBot:")
    response = rag_answer(user_question)
    print(response)
    print("\n" + "-" * 60 + "\n")


Enterprise Support Chatbot is running.
Type your question. Type 'exit' to stop.


Bot:
Answer based on company documents:

Context:
5. Refund Policy
Refunds are processed within 5–7 business days after approval.
Approved refunds are credited back to the original payment method.

GENERAL SUPPORT

6. Updating Personal Information
Users can update their name, address, and contact details from the Profile section.

BILLING & PAYMENTS

4. Payment Failure
Payments may fail due to insufficient balance, expired cards, or bank restrictions.
Users are advised to verify card details or contact their bank.

7. Contacting Support
Customers can contact support via email or live chat.
Support is available Monday to Friday, 9 AM to 6 PM.

Question:
How do I reset my password?

Final Answer:
Based on the provided documentation, please follow the relevant steps described above.

------------------------------------------------------------


Bot:
Answer based on company documents:

Context:
7. Contacting

KeyboardInterrupt: Interrupted by user

In [ ]:
rag_answer("What is the CEO salary?")


"Answer based on company documents:\n\nContext:\nBILLING & PAYMENTS\n\n4. Payment Failure\nPayments may fail due to insufficient balance, expired cards, or bank restrictions.\nUsers are advised to verify card details or contact their bank.\n\n5. Refund Policy\nRefunds are processed within 5–7 business days after approval.\nApproved refunds are credited back to the original payment method.\n\nGENERAL SUPPORT\n\n6. Updating Personal Information\nUsers can update their name, address, and contact details from the Profile section.\n\nACCOUNT & LOGIN SUPPORT\n\n1. Password Reset\nUsers can reset their password by navigating to the Account Settings page.\nClick on 'Forgot Password', enter the registered email address, and follow the reset link sent via email.\n\nQuestion:\nWhat is the CEO salary?\n\nFinal Answer:\nBased on the provided documentation, please follow the relevant steps described above."

In [ ]:
from fastapi import FastAPI
from pydantic import BaseModel


In [ ]:
class ChatRequest(BaseModel):
    question: str


In [ ]:
app = FastAPI(
    title="Enterprise Customer Support GenAI Chatbot",
    description="RAG-based chatbot using FAISS and document grounding",
    version="1.0"
)


In [ ]:
@app.post("/chat")
def chat(request: ChatRequest):
    """
    Accepts a user question and returns a RAG-based answer.
    """
    answer = rag_answer(request.question)
    return {
        "question": request.question,
        "answer": answer
    }
